In [ ]:
import sys, os
import numpy as np
import mdtraj as md

In [ ]:
## pbc treatment of aligned trajectories##

for i in range(20):
    t = md.load('%d/aligned_%d.xtc'%(i,i),top='new_test.pdb')  # load aligned trajectory
    new_t = np.zeros((t.xyz.shape))
    for j in range(t.xyz.shape[0]):         # t.xyz.shape[0] is the number of frames (2000001)
        for k in range(t.xyz.shape[1]):     # t.xyz.shape[1] is the number of atoms (0-11)
            for l in range(t.xyz.shape[2]): # t.xyz.shape[2] is the number of coordinates (x,y,z)
                c = t.xyz[j][k][l]          # load that coordinate (x/y/z)
                if c > 3.00859/2.:          # treat pbc
                    c - 3.00859/2.
                elif c < -3.00859/2.:
                    c + 3.00859/2.
                else:
                    c = c
                new_t[j][k][l] = c          # assign new coordinates
    np.save('%d/aligned_pbc_treat.npy'%i,new_t)     # save new xyz for the trajectory


In [ ]:
## compute spherical coordinates ##

for i in range(20):
    print 'traj', i
    xyz = np.load('%d/aligned_pbc_treat.npy'%i)   # load new xyz coordinates
    n_frames = xyz.shape[0]    # number of frames
    spherical = np.zeros((n_frames,4)) # we only need r,cos_theta, cos_phi, sin_phi  for the ligand
    for j in range(n_frames):
        if xyz[j][11][0] == 0. and xyz[j][11][1] == 0. and xyz[j][11][2] == 0.:    # although unlikely, if the ligand is on the original point exactly
            r = 0.
            cos_theta_r = 0.
            cos_phi_r = 0.
            sin_phi_r = 0.
        else:
            r = np.sqrt(xyz[j][11][0]**2.+xyz[j][11][1]**2.+xyz[j][11][2]**2.)   # r = sqrt(x**2. + y**2. + z**2.)
            cos_theta = xyz[j][11][2]/r                      # cos_theta = z/r
            if xyz[j][11][0] == 0. and xyz[j][11][1] == 0.:     # if the ligand is on the z axis
                cos_phi = 0.                
                sin_phi = 0.
            else:
                cos_phi = xyz[j][11][0]/np.sqrt(xyz[j][11][0]**2.+xyz[j][11][1]**2.)   # cos_phi = x/sqrt(x**2.+y**2.)
                sin_phi = xyz[j][11][1]/np.sqrt(xyz[j][11][0]**2.+xyz[j][11][1]**2.)    # sin_phi = y/sqrt(x**2.+y**2.)

        spherical[j][0] = r
        spherical[j][1] = cos_theta
        spherical[j][2] = cos_phi
        spherical[j][3] = sin_phi
    np.save('new_r2/spher_%d.npy'%i,spherical)


In [ ]:
## find frame index where ligand moves out of the box ##
r_ind=[[] for i in range(20)]
for i in range(20):
    print 'traj', i
    xyz = np.load('%d/aligned_pbc_treat.npy'%i)   
    n_frames = xyz.shape[0]
    for j in range(n_frames):
        r = np.sqrt(xyz[j][11][0]**2.+xyz[j][11][1]**2.+xyz[j][11][2]**2.)
        if r >= 3.00859/2.:
            r_ind[i].append(j)
np.save('r_ind.npy',r_ind)

## group the frame index based on the consecutive integers##

from itertools import groupby
from operator import itemgetter


r_ind = np.load('r_ind.npy')
r_ind_dvd=[[] for i in range(20)]
for i in range(20):
    print 'traj',i
    for k, g in groupby(enumerate(r_ind[i]), lambda (i, x): i-x):
        r_ind_dvd[i].append(list(map(itemgetter(1),g)))
        
np.save('r_ind_dvd.npy',r_ind_dvd)

## get starting and end point for each subset trajectories  ##
ind = np.load('r_ind_dvd.npy')
st_fin = [[] for i in range(20)]
for i in range(len(ind)):
    print 'traj',i
    for j in range(len(ind[i])+1):
        if j == len(ind[i]):
            st_fin[i].append(ind[i][j-1][-1])
            st_fin[i].append(2000001)
        else:
            st_fin[i].append(ind[i][j][0])
            st_fin[i].append(ind[i][j][-1])
np.save('st_fin.npy',st_fin)


In [ ]:
## divide the original trajectories into subsets ##

ind = np.load('st_fin.npy')

for i in range(20):
    print 'traj',i
    r = np.load('new_r2/spher_%d.npy'%i)   # load pre-computed spherical coordinates
    traj = []
    for j in range(len(ind[i])-1):
        print j
        traj.append(r[ind[i][j]:ind[i][j+1]])   # append coordinates for each subset trajectories
 
    np.save('new_r2/divide/traj_%d.npy'%i,traj)


In [ ]:
### build tICA ###


from msmbuilder.featurizer import DihedralFeaturizer
from msmbuilder.dataset import dataset
import numpy as np
from matplotlib import pyplot as plt
import mdtraj as md
import os,sys, glob
import msmbuilder.utils
import pickle
from msmbuilder.utils import load
from msmbuilder.cluster import KMeans
from msmbuilder.cluster import KCenters
from msmbuilder.cluster import KMedoids
from msmbuilder.cluster import MiniBatchKMeans
from msmbuilder.msm import implied_timescales
from msmbuilder.msm import ContinuousTimeMSM, MarkovStateModel
from itertools import combinations
from msmbuilder.featurizer import AtomPairsFeaturizer
from msmbuilder.decomposition import tICA
from sklearn.pipeline import Pipeline
from msmbuilder.example_datasets import fetch_met_enkephalin
from matplotlib import pyplot as plt
from sklearn.externals import joblib


distances=[]

for i in range(20):
        print i
        b=np.load('../../new_r2/divide/traj_%d.npy'%i)
        for j in range(len(b)):
            distances.append(b[j])

tica_model = tICA(lag_time=1,n_components=4)
tica_fit = tica_model.fit(distances)
output=open('tica_model.npy','wb')
pickle.dump(tica_model,output)
output.close()
tica_transformed = tica_fit.transform(distances)
output=open('tica_eigenvectors.npy','wb')
pickle.dump(tica_model.eigenvectors_,output)
output.close()

np.save('tica.npy',tica_transformed)

